In [22]:
import time

from pydantic import BaseModel
from openai import OpenAI
client = OpenAI()


In [2]:
    
def get_res(model,role,task,data):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "developer", "content": "あなたはエコノミストの下で働く有能なアシスタントです。"+role},
            {
                "role": "user",
                "content": task+data
            }
        ]
    )

    return completion.choices[0].message

In [32]:

class YourAnswer(BaseModel):
    employment_risk:list[str]
    inflation_risk:list[str]
    future_monetary_policy:list[str]

def formatted_res(model,role,task,data):
    completion = client.beta.chat.completions.parse(
        model=model,
        messages=[
            {"role": "developer", "content": "あなたはエコノミストの下で働く有能なアシスタントです。"+role},
            {
                "role": "user",
                "content": task+data
            }
        ],
        response_format=YourAnswer,
    )

    return completion.choices[0].message.parsed

In [7]:
model = 'o3-mini'

In [8]:
role = '''
米金融政策を分析する仕事に就いています。
'''

In [33]:
task = '''
データは直近と前回のFOMC議事要旨から抜粋したテキストです。
# 依頼内容
前回と比較して、直近のFOMCにおける議論の特徴を教えてください。
--雇用リスクに関する議論の特徴
--インフレリスクに関する議論の特徴

# 分析上の着目点
-- 意見を述べている参加者の数の違いに着目し、多数派の意見、少数派の意見、コンセンサスを明確に分けること
-- 前回と直近の共通点と相違点を明確に分析すること
-- 前回と直近の議論を踏まえて、今後の金融政策のスタンスについて分析し、利下げ、据え置き、利上げのいずれかの見通しとその理由を示すこと

#データ
'''

In [11]:
txt={}
with open('202409_minutes.txt',encoding='utf-8',mode='r') as f:
    txt['202409']=[{'para':i,'discussion':s} for i,s in enumerate(f.read().split('\n'))]

with open('202412_minutes.txt',encoding='utf-8',mode='r') as f:
    txt['202412']=[{'para':i,'discussion':s} for i,s in enumerate(f.read().split('\n'))]

In [18]:
import tiktoken

encoding = tiktoken.encoding_for_model('gpt-4o')

In [19]:
len(encoding.encode(str(txt)))

5026

In [34]:
start = time.time()
res = get_res(model,role,task,str(txt))
print(f'{time.time()-start:.1f}')

31.0


In [35]:
start = time.time()
res_formatted = formatted_res(model,role,task,str(txt))
print(f'{time.time()-start:.1f}')

17.7


In [36]:
print(res.content)

以下、直近（202412）と前回（202409）のFOMC議事要旨の内容を比較しながら、議論の特徴について分析します。

──────────────────────────────
【１．雇用リスクに関する議論の特徴】

＜前回（202409）の議論＞
• 多くの参加者が、景気拡大・堅調な雇用増加とともに、パンデミック直前に比べて労働市場のタイトさは緩和していると認識していました。  
• 労働市場のソフト化（賃金伸びの鈍化、求人件数の減少、離職率の低下など）の兆候が確認されたものの、依然として全体的には強さを維持しており、「最大雇用」という長期目標にほぼ沿っているとのコンセンサスがありました。  
• 一部参加者は、緩やかな軟化は続いているものの、過度な冷え込みには慎重な立場を示し、今後の動向を注視する必要があるとの見方もありました。

＜直近（202412）の議論＞
• 労働市場に関しても、求人減少や回転率の低下といったソフト化の兆候が強調されました。データからは「漸次緩和」が進んでいるとの認識が共有されています。  
• しかし、参加者のほとんどは、現状は依然として最大雇用の状態に近いとの見解を示す一方、今後の不確実性（たとえば、基調となる成長率の変動や雇用統計の改定など）から、慎重な評価が必要であるとの意見もあり、雇用側には下方リスクがあるというリスク認識が示されました。  

＜比較・まとめ＞
・前回は「労働市場は十分に堅調で、さらなる過熱感は見受けられず、なおかつ適度なソフト化が進んでいる」という評価が圧倒的多数でした。  
・直近では、依然として最大雇用に近いとしつつも、最近のデータからはソフト化の兆候が顕著になっており、下方リスクを考慮する必要性が強調されたため、参加者間でも見解の幅が生じた印象です。

──────────────────────────────
【２．インフレリスクに関する議論の特徴】

＜前回（202409）の議論＞
• インフレに関しては、参加者のほぼ全員が「最近のデータやビジネスからの情報に基づき、インフレは2％に向け持続的な軌道にある」と認識していました。  
• さらに、食品・エネルギー価格以外でも、コア財・サービス価格全般の伸びが鈍化していることを根拠に、今後の下方圧力（実質GDPの緩やかな減速、価格設定力の低下、労

In [37]:
[print(s,'\n') for s in res_formatted.employment_risk]
print('***************')
[print(s,'\n') for s in res_formatted.inflation_risk]
print('***************')
[print(s,'\n') for s in res_formatted.future_monetary_policy]

【共通点】両会合とも、労働市場は堅調ながらも一部でやや軟化の兆候が見られ、ペースの鈍化や失業率の上昇といった点で注意が払われている。 

【202409会合】多数派の参加者は、パンデミック前より労働市場が緩和していると評価しながらも、引き続き雇用の安定を強調。少数派は、過度な労働市場の冷え込みへのリスクについて触れ、今後のさらなる緩和が深刻な悪化につながる可能性を示唆。 

【202412会合】大多数は、労働市場は依然として堅調であり、緩やかな軟化を示すものの急激な悪化は見込まれないと判断。少数意見では、既存のペースが失業率維持に十分でない可能性や慎重な見極めの必要性が指摘される。 

***************
【共通点】両会合とも、インフレ率は2%目標への持続的な接近が示唆される一方、特定の品目（特に住宅関連）における頑固な高水準が懸念材料として挙げられている。 

【202409会合】多数派は、広範なコア指標における物価低下が進んでいると楽観的に評価し、下方リスクを強調。一部の参加者は、食品・エネルギー以外でも持続的な下方圧力が働いていると評価。 

【202412会合】参加者の大半は、ディスインフレーションの進展はあるものの、直近のデータで一部上昇圧力が見られる点を懸念。多数は、貿易や移民政策の変化、さらには持続的なシェルター価格の上昇が、インフレの上方リスクを高めると指摘している。 

***************
【202409会合】多数派が50ベーシスポイントの利下げを支持し、インフレと雇用のリスクを均衡させるため、より緩和的な金融政策スタンスへの転換が示唆された。 

【202412会合】多数派は25ベーシスポイントの利下げを選択し、インフレリスクの上昇を背景に、より慎重・段階的な政策正常化の道筋が強調された。各参加者は、今後の経済指標に応じた条件付きで中立的な政策スタンスへの移行を視野に入れている。 



[None, None]